In [1]:
from model import Deeplabv3
model=Deeplabv3(weights='pascal_voc',input_shape=(320,320,3),classes=21)

Using TensorFlow backend.


In [2]:
from keras import optimizers
# model=FCN_Vgg16_32s(input_shape=(320,320,3),classes=21)
optimizer=optimizers.RMSprop(lr=1e-5)
#可以自定义损失函数和metrics
# loss_fn=softmax_sparse_crossentropy_ignoring_last_label
# metrics=[sparse_accuracy_ignoring_last_label]
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['acc'])
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 160, 160, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv_BN (BatchNormalization)    (None, 160, 160, 32) 128         Conv[0][0]                       
__________________________________________________________________________________________________
Conv_Relu6 (Activation)         (None, 160, 160, 32) 0           Conv_BN[0][0]                    
__________________________________________________________________________________________________
expanded_c

In [3]:
import numpy as np
colormap = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0], [0, 0, 128],
            [128, 0, 128], [0, 128, 128], [128, 128, 128], [64, 0, 0], [192, 0, 0],
            [64, 128, 0], [192, 128, 0], [64, 0, 128], [192, 0, 128],
            [64, 128, 128], [192, 128, 128], [0, 64, 0], [128, 64, 0],
            [0, 192, 0], [128, 192, 0], [0, 64, 128]]
cm2lbl = np.zeros(256 ** 3)  # 每个像素点有 0 ~ 255 的选择，RGB 三个通道
for i, cm in enumerate(colormap):
    cm2lbl[(cm[0] * 256 + cm[1]) * 256 + cm[2]] = i  # 建立索引


def image2label(im):
    # 输入三维的mask图片，返回二维的矩阵，对每一个像素进行标记(320,320,3)->(320,320)
    #如[0,0,0]标记为0,[128,0,0]标记为1
    data = np.array(im, dtype='int32')
    idx = (data[:, :, 0] * 256 + data[:, :, 1]) * 256 + data[:, :, 2]
    return np.array(cm2lbl[idx], dtype='int64')


def adjustData(img, mask, num_class):
    # 将mask转换为one_hot形式(320,320)->(320,320,21)
    img, _ = img
    mask, _ = mask
    
    img = img / 255.
    mask_label = np.zeros(mask[:, :, :, 0].shape)
    # 将3通道的mask首先转换为二维的label
    for j in range(mask.shape[0]):
        mask_label[j] = image2label(mask[j])

    mask_one_hot = np.zeros(mask_label.shape + (num_class,))
    # 将二维的label转换为三维的one_hot形式
    for i in range(num_class):
        mask_one_hot[mask_label == i, i] = 1
    mask = mask_one_hot.astype('int32')
    return (img, mask)

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
# train_datagen=ImageDataGenerator()

checkpoint=ModelCheckpoint('./models/lab_01_deeplab_v3.hdf5',monitor='val_acc',save_best_only=True,verbose=1)
def train_generator(data_gen_args):
    train_images_datagen = ImageDataGenerator(**data_gen_args)
    train_masks_datagen = ImageDataGenerator(**data_gen_args)

    train_images_generator=train_images_datagen.flow_from_directory('/input0/pascal_2012_seg/train',classes=['images'],class_mode='categorical',seed=1,target_size=(320,320),batch_size=20)
    train_tasks_generator=train_masks_datagen.flow_from_directory('/input0/pascal_2012_seg/train',classes=['masks'],class_mode='categorical',seed=1,target_size=(320,320),batch_size=20)
    train_generator=zip(train_images_generator,train_tasks_generator)

    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,21)
        yield (img,mask)

def val_generator():
    val_images_datagen = ImageDataGenerator()
    val_masks_datagen = ImageDataGenerator()

    val_images_generator=val_images_datagen.flow_from_directory('/input0/pascal_2012_seg/validation',classes=['images'],class_mode='categorical',seed=1,target_size=(320,320),batch_size=20)
    val_tasks_generator=val_masks_datagen.flow_from_directory('/input0/pascal_2012_seg/validation',classes=['masks'],class_mode='categorical',seed=1,target_size=(320,320),batch_size=20)
    val_generator=zip(val_images_generator,val_tasks_generator)
    for (img,mask) in val_generator:
        img,mask = adjustData(img,mask,21)
        yield (img,mask)
        

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    horizontal_flip=True,
                    fill_mode='constant',cval=0)
#需要数据增强的时候直接注释掉下一行就行
data_gen_args=dict()
train_generator = train_generator(data_gen_args)
val_generator=val_generator()
history=model.fit_generator(train_generator,steps_per_epoch=120,epochs=30,validation_data=val_generator,validation_steps=15,callbacks=[checkpoint])


Epoch 1/30
Found 2330 images belonging to 1 classes.
Found 290 images belonging to 1 classes.
Found 2330 images belonging to 1 classes.
Found 290 images belonging to 1 classes.
120/120 [==============================] - 136s 1s/step - loss: 0.2558 - acc: 0.9087 - val_loss: 0.2589 - val_acc: 0.9076

Epoch 00001: val_acc improved from -inf to 0.90758, saving model to ./models/lab_01_deeplab_v3.hdf5
Epoch 2/30
120/120 [==============================] - 110s 920ms/step - loss: 0.2124 - acc: 0.9224 - val_loss: 0.2371 - val_acc: 0.9155

Epoch 00002: val_acc improved from 0.90758 to 0.91549, saving model to ./models/lab_01_deeplab_v3.hdf5
Epoch 3/30
120/120 [==============================] - 119s 995ms/step - loss: 0.1975 - acc: 0.9282 - val_loss: 0.2279 - val_acc: 0.9186

Epoch 00003: val_acc improved from 0.91549 to 0.91863, saving model to ./models/lab_01_deeplab_v3.hdf5
Epoch 4/30
120/120 [==============================] - 115s 955ms/step - loss: 0.1878 - acc: 0.9319 - val_loss: 0.2253 - 